In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
from src.database import VastDB
from src.manager import DbManager
from src.fetch import fetch
from src.preprocess import preprocess, split_raw
from src import const
import logging


logging.basicConfig(format=const.LOG_FORMAT,
                    level=logging.DEBUG,
                    datefmt='%d-%m-%Y %I:%M:%S')

# logger = 
# logger.setLevel(logging.DEBUG)

In [3]:
db = VastDB('../data/test.db')
dbm = db.dbm
db.connect()

[06-10-2023 06:20:29] [INFO] Connected to the database '../data/test.db'


In [4]:
dbm.get_tables()

['tmp_machines',
 'tmp_offers',
 'host_mach_map',
 'mach_offer_map',
 'chunks_map',
 'timestamp_idx']

In [5]:
info = dbm.get_tbl_info('tmp_machines')
info
# 'host_id' in info['name']

,name,type,notnull,dflt_value,pk
cid,,,,,
0,bw_nvlink,INTEGER,0,None,0
1,client_run_time,REAL,0,None,0
2,compute_cap,INTEGER,0,None,0
3,cpu_cores,INTEGER,0,None,0
4,cpu_name,TEXT,0,None,0
5,cpu_ram,INTEGER,0,None,0
6,credit_discount_max,INTEGER,0,None,0
7,cuda_max_good,TEXT,0,None,0
8,direct_port_count,INTEGER,0,None,0


In [6]:
db.create_map_tables()

In [7]:
db.dbm.get_tables()

['tmp_machines',
 'tmp_offers',
 'host_mach_map',
 'mach_offer_map',
 'chunks_map',
 'timestamp_idx']

In [8]:
for tbl in db.dbm.get_tables():
    db.dbm.delete_table(tbl)

[06-10-2023 06:20:30] [INFO] Table 'tmp_machines' deleted
[06-10-2023 06:20:30] [INFO] Table 'tmp_offers' deleted
[06-10-2023 06:20:30] [INFO] Table 'host_mach_map' deleted
[06-10-2023 06:20:30] [INFO] Table 'mach_offer_map' deleted
[06-10-2023 06:20:30] [INFO] Table 'chunks_map' deleted
[06-10-2023 06:20:30] [INFO] Table 'timestamp_idx' deleted


In [9]:
db.dbm.create_table('tmp_machines', ['host_id', 'machine_id', 'timestamp'])
db.dbm.insert('tmp_machines',
              [[1, 2, 16],
               [1, 3, 16],
               [2, 4, 16]])

[06-10-2023 06:20:30] [INFO] Table 'tmp_machines' created
[06-10-2023 06:20:30] [INFO] 3 Rows inserted into 'tmp_machines'


In [10]:
db.dbm.create_table('tmp_offers', ['machine_id', 'id', 'num_gpus', 'timestamp'])
db.dbm.insert('tmp_offers', 
                  [[1, 1, 1, 16], 
                   [1, 2, 1, 16], 
                   [2, 3, 2, 16]])

[06-10-2023 06:20:30] [INFO] Table 'tmp_offers' created
[06-10-2023 06:20:30] [INFO] 3 Rows inserted into 'tmp_offers'


In [11]:
db.create_map_tables()
db.update_tables()
# db.dbm.commit()
db.dbm.get_db_stats()

,name,pages,size,percentage
0,total,6,0.02,100.00
1,sqlite_schema,1,0.00,19.53
2,tmp_machines,1,0.00,19.53
3,tmp_offers,1,0.00,19.53
4,host_mach_map,1,0.00,19.53
5,mach_offer_map,1,0.00,19.53
6,chunks_map,1,0.00,19.53


In [12]:
# db.dbm.table_to_df('host_mach_map')
# db.dbm.table_to_df('mach_offer_map')
db.dbm.table_to_df('chunks_map')

,id,num_gpus,timestamp
0,1,1,16
1,2,1,16
2,3,2,16


In [13]:
db.update_tables()

0

In [52]:
# db.connect()

for tbl in db.dbm.get_tables():
    db.dbm.delete_table(tbl)

db.close()

[06-10-2023 06:27:07] [INFO] Table 'tmp_machines' deleted
[06-10-2023 06:27:07] [INFO] Table 'tmp_offers' deleted
[06-10-2023 06:27:07] [INFO] Table 'host_mach_map' deleted
[06-10-2023 06:27:07] [INFO] Table 'mach_offer_map' deleted
[06-10-2023 06:27:07] [INFO] Table 'chunks_map' deleted
[06-10-2023 06:27:07] [INFO] Table 'timestamp_idx' deleted
[06-10-2023 06:27:07] [INFO] Disconnected from the database '../data/test.db'.


In [97]:
raw = fetch()
preprocess(raw)
machines, offers = split_raw(raw)

[06-10-2023 06:38:25] [INFO] [API] fetching data from '500.farm'
[06-10-2023 06:38:25] [DEBUG] Starting new HTTPS connection (1): 500.farm:443
[06-10-2023 06:38:27] [DEBUG] https://500.farm:443 "GET /vastai-exporter/offers HTTP/1.1" 200 None


In [98]:
db.connect()
db.create_tmp_tables(machines, offers)
db.create_map_tables()

[06-10-2023 06:38:28] [INFO] Connected to the database '../data/test.db'


In [99]:
db.dbm.get_db_stats()
# db.dbm.delete_table('int32')

,name,pages,size,percentage
0,total,49,0.19,100.00
1,mach_offer_map,20,0.08,41.12
2,chunks_map,18,0.07,37.01
3,host_mach_map,9,0.04,18.50
4,sqlite_schema,1,0.00,2.06
5,timestamp_idx,1,0.00,2.06


In [100]:
db.create_ts_index()

In [101]:
db.update_tables()

0

In [102]:
db.dbm.commit()

In [103]:
db.dbm.get_db_stats()

,name,pages,size,percentage
0,total,49,0.19,100.00
1,mach_offer_map,20,0.08,41.12
2,chunks_map,18,0.07,37.01
3,host_mach_map,9,0.04,18.50
4,sqlite_schema,1,0.00,2.06
5,timestamp_idx,1,0.00,2.06


In [104]:
# db.dbm.table_to_df('mach_offer_map')
db.dbm.table_to_df('mach_offer_map').sort_values(by='machine_id')

,id,machine_id,timestamp
0,6612968,162,1696566579
1,6363233,274,1696566579
2,6363229,274,1696566579
3,6363228,274,1696566579
4,6363227,274,1696566579
...,...,...,...
4246,7177116,13757,1696566579
4247,7177145,13758,1696566579
4248,7177293,13760,1696566579
4249,7177800,13761,1696566579


In [105]:
db.dbm.table_to_df('chunks_map').sort_values(by='id')

,id,num_gpus,timestamp
429,5381479,1,1696566579
428,5381480,1,1696566579
427,5381481,1,1696566579
426,5381484,1,1696566579
413,5393995,8,1696566579
...,...,...,...
1325,7177591,1,1696566579
430,7177597,2,1696566579
2791,7177599,1,1696566579
4249,7177800,1,1696566579


In [106]:
db.dbm.table_to_df('tmp_offers').sort_values(by='id')

,bw_nvlink,client_run_time,compute_cap,cpu_cores,cpu_name,cpu_ram,credit_discount_max,cuda_max_good,direct_port_count,disk_bw,...,reliability,rented,score,static_ip,storage_cost,total_flops,verification,vram_costperhour,timestamp,country
429,60,1.1,860,128,,252,400,11.8,500,1283,...,9990,1,61,0,150,30,1,0.000009,1696567050,US
428,60,1.1,860,128,,252,400,11.8,500,1283,...,9990,1,58,0,150,30,1,0.000009,1696567050,US
427,60,1.1,860,128,,252,400,11.8,500,1283,...,9990,1,57,0,150,30,1,0.000009,1696567050,US
426,60,1.1,860,128,,252,400,11.8,500,1283,...,9990,1,61,0,150,30,1,0.000009,1696567050,US
413,0,1.1,860,128,AMD EPYC 7502 32-Core Processor,252,0,11.7,100,3366,...,9996,1,50,1,70,282,1,0.000018,1696567050,RU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,0,1.1,890,16,AMD Ryzen 7 5700X 8-Core Processor,63,400,12.0,100,3162,...,9988,1,161,1,200,81,1,0.000016,1696567050,MO
430,0,1.1,860,28,Xeon® E5-2680 v4,63,400,12.0,100,812,...,9990,1,162,1,400,71,1,0.000007,1696567050,MO
2791,0,1.1,890,16,AMD Ryzen 7 5700X 8-Core Processor,31,400,12.0,100,4063,...,9975,1,149,1,200,81,1,0.000016,1696567050,MO
4249,0,1.1,890,48,AMD EPYC 7402 24-Core Processor,252,0,12.2,1000,0,...,5727,0,0,0,100,81,0,0.000008,1696567050,


In [107]:
db.close()

[06-10-2023 06:38:29] [INFO] Disconnected from the database '../data/test.db'.
